In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import dateutil.parser
from datetime import date
import re

from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import time

plt.style.use("seaborn")
%matplotlib inline

%load_ext autotime

In [7]:
link='https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?skuId=1925486&icid2=products%20grid:p87985432'

time: 722 µs


In [4]:
cols = ['user', 'rating', 'helpfulness', 'skin_color', 'skin_concerns', 'review', 'review_date']
df=pd.DataFrame(columns=cols, index=[])

time: 16.9 ms


In [9]:
response=requests.get(link)
response.status_code

200

time: 3.66 s


In [10]:
soup = BeautifulSoup(response.text, 'html5lib')

time: 196 ms


In [11]:
from IPython.core.display import display, HTML
display(HTML(response.text))

time: 16.6 ms


In [ ]:
age=date.today()
for link in links:
    response=requests.get(link)
    soup = BeautifulSoup(response.text, 'html5lib')
    try: 
        main_search_table=soup.find('div', id='search_result_container').find_all('div')[1]
    except:
        time.sleep(20)
        main_search_table=soup.find('div', id='search_result_container').find_all('div')[1]
    print(link)
    for game in main_search_table.find_all('a'):
        row={}
        row['title']=game.find('span', class_="title").text #title
        row['link']=game['href'] #link
        systems=[system["class"][1] for system in game.find('p').find_all('span')] #system
        row['system']=len(systems)
        row['systems']=systems
        try:
            row['release_date']=(age-(to_date(game.find('div', class_="col search_released responsive_secondrow").text).date())).days #release date  
        except:
            row['release_date']=0
        for price in game.find_all('div', class_="col search_price_discount_combined responsive_secondrow"): #price and discount
            for discount in price.find_all('div', class_="col search_discount responsive_secondrow"):
                if discount.find('span'):
                    row['discount']=abs(int(discount.find('span').text.strip('%')))
                    try:
                        row['price']=float(price.find('span', style="color: #888888;").text.strip("$"))
                    except:
                        price = re.sub('[^0-9,.]','', game.find('div', class_="col search_price responsive_secondrow").text).replace("\n","").strip("\t")
                        row['price']=price
                else:
                    row['discount']=0
                    if not game.find('div', class_="col search_price responsive_secondrow").text.replace("\n","").strip("\t"):
                        row['price']=0
                    elif game.find('div', class_="col search_price responsive_secondrow").text.replace("\n","").strip("\t").lower()[0]!='$': #'free to play' or game.find('div', class_="col search_price responsive_secondrow").text.replace("\n","").strip("\t").lower()=='free':
                        row['price']=0
                    else:
                        row['price']=float(game.find('div', class_="col search_price responsive_secondrow").text.replace("\n","").strip("\t").strip("$"))  
        for y in game.find_all('div',class_="col search_reviewscore responsive_secondrow"):
            x=[review['data-tooltip-html'] for review in y.find_all('span')]
            if not x:
                row['reviews']=int(0)
            else:
                z=x[0].split('<br>')
                d=z[1].split(" ")
                row['reviews']=int(d[3].replace(',', '')) #reviews
        df=df.append(row, ignore_index=True)
        with open('steam_search.pkl', 'wb') as picklefile:
            pickle.dump(df, picklefile)
#     time.sleep(3)